In [203]:
#from implementations import *
from project_helpers import *
import csv
import numpy as np
import pandas as pd
import math

In [204]:
def AIC_forward(y, x):
    
    left = set(range(1, x.shape[1]))
    picked = [0]
    
    current, new = 1000000.0, 1000000.0
    
    while left and current == new:
        
        aics_cov = []
        
        for covariate in left:
            columns = picked + [covariate]
            loss = least_squares(y, x[:,columns])[1]
            aic = 2*loss*y.shape[0] + 2*len(columns)
            aics_cov.append((aic, covariate))
        
        aics_cov.sort()
        new, best_cov = aics_cov[0]
        
        if current > new:
            left.remove(best_cov)
            picked.append(best_cov)
            current = new
            
    return picked

In [205]:
def split_data(x, y, ratio=0.8, seed=1):
    """split the dataset based on the split ratio."""
    # set seed
    np.random.seed(seed)
    # generate random indices
    num_row = len(y)
    indices = np.random.permutation(num_row)
    index_split = int(np.floor(ratio * num_row))
    index_tr = indices[: index_split]
    index_te = indices[index_split:]
    # create split
    x_tr = x[index_tr]
    x_te = x[index_te]
    y_tr = y[index_tr]
    y_te = y[index_te]
    return x_tr, x_te, y_tr, y_te

In [206]:
def build_poly(x, degree):
    """polynomial basis functions for input data x, for j=0 up to j=degree."""
    poly = np.ones((len(x), 1))
    for deg in range(1, degree+1):
        poly = np.c_[poly, np.power(x, deg)]
    return poly

In [207]:
def compute_mse(y, tx, w):
    """compute the loss by mse."""
    e = y - tx.dot(w)
    mse = e.dot(e) / (2 * len(e))
    return mse

In [208]:
def ridge_regression(y, tx, lambda_):
    lambda_prime = (lambda_*2*y.shape[0]) * np.identity(tx.shape[1])
    a = tx.T.dot(tx) + lambda_prime
    b = tx.T.dot(y)
    return np.linalg.solve(a, b)

In [209]:
def sigmoid(z):
    #return 1 / (1+np.exp(-z))
    return .5 * (1 + np.tanh(.5 * z))

def predict(w, tx, threshold=0.5):
    probs = sigmoid(np.dot(tx, w))
    probs[probs>=threshold] = 1
    probs[probs<threshold] = 0
    return probs

def logistic_regression(y, tx, initial_w, max_iters, gamma):
   
    w = initial_w
    h = 0
    
    for i in range(max_iters):
    
        #Compute x_t*w
        z = np.dot(tx, w)
        #Compute sigmoid of z
        h = sigmoid(z)

        #Compute gradient
        gradient = tx.T.dot(h-y)
        
        #Update weight vector
        update = gamma*gradient
        w = w - update
        
    return w

In [210]:
def model_pick_ridge(x, y, ratio=0.8, seed=1, degrees=range(1,2), lambdas=range(1)):
    """Pick best polynomial basis expansion and ridge regression lambda based on cross
    validation mse scores"""

    # split data
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)

    # ridge regression with different basis degrees and lambdas
    rmse_tr = []
    rmse_te = []
    
    for degree in degrees:
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        for lambda_ in lambdas:
            weight = ridge_regression(y_tr, tx_tr, lambda_)
            rmse_tr.append((np.sqrt(2 * compute_mse(y_tr, tx_tr, weight)), degree, lambda_))
            rmse_te.append((np.sqrt(2 * compute_mse(y_te, tx_te, weight)), degree, lambda_))
            
    rmse_tr.sort()
    rmse_te.sort()
    return rmse_tr, rmse_te

In [211]:
def model_pick_logistic(x, y, ratio=0.8, seed=1, degrees=range(1,2), lambdas=range(1)):
    """Pick best polynomial basis expansion and regularized logistic regression lambda 
    based on cross validation mse scores"""

    # split data
    x_tr, x_te, y_tr, y_te = split_data(x, y, ratio, seed)

    # ridge regression with different basis degrees and lambdas
    e_tr = []
    e_te = []
    
    for degree in degrees:
        tx_tr = build_poly(x_tr, degree)
        tx_te = build_poly(x_te, degree)
        for lambda_ in lambdas:
            weight = logistic_regression(y_tr, tx_tr, np.zeros(tx_tr.shape[1]),1000,0.001)
            pred_tr = predict(weight, tx_tr)
            pred_te = predict(weight, tx_te)
            score_tr = (pred_tr == y_tr).mean()
            score_te = (pred_te == y_te).mean()
            e_tr.append((score_tr, degree, lambda_))
            e_te.append((score_te, degree, lambda_))
            
    e_tr.sort(reverse=True)
    e_te.sort(reverse=True)
    return e_tr, e_te

Exploratory Data Analysis:

In [212]:
#Load train data
x_pd = pd.read_csv('Data/train.csv')
x_pd.head()

,Id,Prediction,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
0,100000,s,138.470,51.655,97.827,27.980,0.91,124.711,2.666,3.064,...,-0.277,258.733,2,67.435,2.150,0.444,46.062,1.24,-2.475,113.497
1,100001,b,160.937,68.768,103.235,48.146,-999.00,-999.000,-999.000,3.473,...,-1.916,164.546,1,46.226,0.725,1.158,-999.000,-999.00,-999.000,46.226
2,100002,b,-999.000,162.172,125.953,35.635,-999.00,-999.000,-999.000,3.148,...,-2.186,260.414,1,44.251,2.053,-2.028,-999.000,-999.00,-999.000,44.251
3,100003,b,143.905,81.417,80.943,0.414,-999.00,-999.000,-999.000,3.310,...,0.060,86.062,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000
4,100004,b,175.864,16.915,134.805,16.405,-999.00,-999.000,-999.000,3.891,...,-0.871,53.131,0,-999.000,-999.000,-999.000,-999.000,-999.00,-999.000,0.000


Many columns have missing values. It looks like that missingness depends on the value of PRI_jet_num.  
  
When jet_num=0:  
DER_deltaeta_jet_jet, DER_mass_jet_jet, DER_prodeta_jet_jet, DER_lep_eta_centrality, PRI_jet_leading_pt, PRI_jet_leading_eta, PRI_jet_leading_phi, PRI_jet_subleading_pt, PRI_jet_subleading_eta, PRI_jet_subleading_phi and PRI_jet_all_pt do not contain any information 
  
When jet_num=1:  
DER_deltaeta_jet_jet, DER_mass_jet_jet, DER_prodeta_jet_jet, DER_lep_eta_centrality, PRI_jet_subleading_pt, PRI_jet_subleading_eta, PRI_jet_subleading_phi do not contain any information  
  
When jet_num=2,3 every column contains relevant information  
  
The column DER_mass_MMC has missing values for all jet_num values  
  
Missing values were arbitrarily set to -999. We will set them to 0. Therefore missing covariates won't affect linear combinations.

In [229]:
x_pd.corr().head()

,Id,DER_mass_MMC,DER_mass_transverse_met_lep,DER_mass_vis,DER_pt_h,DER_deltaeta_jet_jet,DER_mass_jet_jet,DER_prodeta_jet_jet,DER_deltar_tau_lep,DER_pt_tot,...,PRI_met_phi,PRI_met_sumet,PRI_jet_num,PRI_jet_leading_pt,PRI_jet_leading_eta,PRI_jet_leading_phi,PRI_jet_subleading_pt,PRI_jet_subleading_eta,PRI_jet_subleading_phi,PRI_jet_all_pt
Id,1.000000,0.001917,-0.006059,-0.001851,0.002073,-0.000060,0.000353,-0.000075,-0.001349,-0.000581,...,-0.001141,0.002312,0.000175,0.001396,0.001216,0.001214,-0.000065,-0.000068,-0.000060,0.001024
DER_mass_MMC,0.001917,1.000000,-0.455755,0.168548,0.198765,0.162661,0.160524,0.162521,0.228105,0.045826,...,0.007411,0.221984,0.221078,0.250158,0.247083,0.247078,0.162836,0.162614,0.162609,0.185372
DER_mass_transverse_met_lep,-0.006059,-0.455755,1.000000,0.190109,-0.249116,-0.176386,-0.190392,-0.175942,0.043251,0.017758,...,-0.015925,-0.167811,-0.210537,-0.229674,-0.220370,-0.220355,-0.176837,-0.176231,-0.176225,-0.210009
DER_mass_vis,-0.001851,0.168548,0.190109,1.000000,-0.062562,-0.032251,-0.040620,-0.032126,0.579712,-0.000702,...,-0.001467,0.053300,-0.026860,-0.019151,-0.013749,-0.013742,-0.033188,-0.032202,-0.032206,-0.052902
DER_pt_h,0.002073,0.198765,-0.249116,-0.062562,1.000000,0.523664,0.534531,0.523639,-0.539379,0.310501,...,0.008585,0.782547,0.623401,0.621599,0.564898,0.564894,0.531647,0.523714,0.523703,0.808616


We observe that many columns are correlated. This may result in an ill-conditioned design matrix when computing inverses.

**Model Building:**  
  
With each approach we will resort to regression, regularization and 80/20 cross validation.

In [214]:
#Load train data
y, tx, ids = load_csv_data('Data/train.csv', sub_sample=False)

In [215]:
tx[tx == -999] = 0

In [216]:
#Define parameters
degrees = range(1,10)
lambdas = [10**-x for x in range(1,14)]

**Approach 1: Ridge regression**  

Building a model:

In [217]:
#Basic linear regression
tr,te = model_pick_ridge(tx, y)
mse,deg,lamb = te[0]
print('mse: ',mse,'deg: ',deg,'lambda: ',lamb)

mse:  0.825104421874223 deg:  1 lambda:  0


In [218]:
#Linear regression with polynomial basis expansion
tr,te = model_pick_ridge(tx, y, degrees=degrees)
mse,deg,lamb = te[0]
print('mse: ',mse,'deg: ',deg,'lambda: ',lamb)

mse:  0.7679063549399294 deg:  6 lambda:  0


In [219]:
#Ridge regression with polynomial basis expansion
tr,te = model_pick_ridge(tx, y, degrees=degrees, lambdas=lambdas)
mse,deg,lamb = te[0]
print('mse: ',mse,'deg: ',deg,'lambda: ',lamb)

mse:  0.7678923926447192 deg:  6 lambda:  1e-06


In [220]:
tx_deg = build_poly(tx, deg)

In [221]:
w = ridge_regression(y, tx_deg, lamb)

Predicting test data:

In [222]:
y_t, tx_t, ids_t = load_csv_data('Data/test.csv', sub_sample=False)

In [223]:
tx_t[tx_t == -999] = 0

In [224]:
tx_t = build_poly(tx_t, deg)
pred = predict_labels(w, tx_t)

**Approach 2: Logistic regression**

Building a model:

In [225]:
#y[y == -1] = 0

In [226]:
#tr,te = model_pick_logistic(tx, y, degrees=degrees)
#score,deg,lamb = te[0]
#print('score: ',score,'deg: ',deg,'lambda: ',lamb)

Predicting test data:

Create Submissions

In [227]:
#create_csv_submission(ids_t, pred, 'pred full model poly expansion ridge reg')